In [2]:
##Decision Tree Classifier for Iris Dataset
import numpy as np
import pandas as pd
import urllib.request
import io

class DecisionTreeClassifier:
    """
    A decision tree classifier implementation using only NumPy and pandas.
    """
    
    def __init__(self, max_depth=None, min_samples_split=2, min_impurity_decrease=0.0):
        """
        Initialize the decision tree classifier.
        
        Parameters:
        -----------
        max_depth : int or None
            Maximum depth of the tree. If None, the tree will expand until all leaves are pure.
        min_samples_split : int
            Minimum number of samples required to split an internal node.
        min_impurity_decrease : float
            A node will be split if this split induces a decrease of the impurity greater than this value.
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.tree = None
        self.feature_names = None
        self.class_names = None
    
    def _gini(self, y):
        """
        Calculate the Gini impurity of a set of labels.
        
        Parameters:
        -----------
        y : array-like
            Array of labels.
            
        Returns:
        --------
        float
            Gini impurity.
        """
        if len(y) == 0:
            return 0
        
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini
    
    def _information_gain(self, y, y_left, y_right):
        """
        Calculate the information gain of a split.
        
        Parameters:
        -----------
        y : array-like
            Array of labels before the split.
        y_left : array-like
            Array of labels in the left branch after the split.
        y_right : array-like
            Array of labels in the right branch after the split.
            
        Returns:
        --------
        float
            Information gain.
        """
        parent_impurity = self._gini(y)
        n = len(y)
        n_left, n_right = len(y_left), len(y_right)
        
        if n_left == 0 or n_right == 0:
            return 0
        
        child_impurity = (n_left / n) * self._gini(y_left) + (n_right / n) * self._gini(y_right)
        return parent_impurity - child_impurity
    
    def _best_split(self, X, y):
        """
        Find the best split for a node.
        
        Parameters:
        -----------
        X : 2d array-like
            The feature data for the node.
        y : array-like
            The target data for the node.
            
        Returns:
        --------
        dict
            Dictionary containing the best feature index, threshold, left indices, and right indices.
        """
        m, n = X.shape
        if m <= self.min_samples_split:
            return None
        
        best_gain = 0.0
        best_split = None
        
        for feature_idx in range(n):
            feature_values = X[:, feature_idx]
            thresholds = np.unique(feature_values)
            
            for threshold in thresholds:
                left_indices = np.where(feature_values <= threshold)[0]
                right_indices = np.where(feature_values > threshold)[0]
                
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                
                y_left, y_right = y[left_indices], y[right_indices]
                gain = self._information_gain(y, y_left, y_right)
                
                if gain > best_gain and gain > self.min_impurity_decrease:
                    best_gain = gain
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'left_indices': left_indices,
                        'right_indices': right_indices,
                        'gain': gain
                    }
        
        return best_split
    
    def _build_tree(self, X, y, depth=0):
        """
        Recursively build the decision tree.
        
        Parameters:
        -----------
        X : 2d array-like
            The feature data.
        y : array-like
            The target data.
        depth : int
            Current depth of the tree.
            
        Returns:
        --------
        dict
            A node of the decision tree.
        """
        # Get the majority class for this node
        unique_classes, counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(counts)]
        
        # Create a leaf node if stopping criteria are met
        if (self.max_depth is not None and depth >= self.max_depth) or len(np.unique(y)) == 1:
            return {'type': 'leaf', 'class': majority_class, 'samples': len(y)}
        
        # Find the best split
        best_split = self._best_split(X, y)
        
        # If no split improves impurity, create a leaf node
        if best_split is None:
            return {'type': 'leaf', 'class': majority_class, 'samples': len(y)}
        
        # Create a decision node
        left_indices = best_split['left_indices']
        right_indices = best_split['right_indices']
        
        # Recursively build the left and right branches
        left_branch = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_branch = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        
        return {
            'type': 'node',
            'feature_idx': best_split['feature_idx'],
            'threshold': best_split['threshold'],
            'gain': best_split['gain'],
            'samples': len(y),
            'left': left_branch,
            'right': right_branch
        }
    
    def fit(self, X, y, feature_names=None, class_names=None):
        """
        Build the decision tree.
        
        Parameters:
        -----------
        X : array-like or pandas DataFrame of shape (n_samples, n_features)
            The training input samples.
        y : array-like or pandas Series of shape (n_samples,)
            The target values.
        feature_names : list, optional
            Names of the features. If None and X is a DataFrame, column names are used.
        class_names : list, optional
            Names of the classes. If None and y is a Series, unique values are used.
            
        Returns:
        --------
        self
            Fitted estimator.
        """
        # Store feature names
        if feature_names is not None:
            self.feature_names = feature_names
        elif isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
        else:
            self.feature_names = [f"feature_{i}" for i in range(X.shape[1])]
        
        # Store class names
        if class_names is not None:
            self.class_names = class_names
        elif isinstance(y, pd.Series):
            self.class_names = y.unique().tolist()
        else:
            self.class_names = [str(c) for c in np.unique(y)]
        
        # Convert pandas DataFrame/Series to numpy arrays if necessary
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        
        # Convert X to 2D array if it's 1D
        if X.ndim == 1:
            X = X.reshape(-1, 1)
            
        # Build the tree
        self.tree = self._build_tree(X, y)
        
        return self
    
    def _predict_one(self, x, node):
        """
        Predict the class for a single sample.
        
        Parameters:
        -----------
        x : array-like
            The input sample.
        node : dict
            The current node in the tree.
            
        Returns:
        --------
        The predicted class.
        """
        if node['type'] == 'leaf':
            return node['class']
        
        if x[node['feature_idx']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])
    
    def predict(self, X):
        """
        Predict classes for samples in X.
        
        Parameters:
        -----------
        X : array-like or pandas DataFrame of shape (n_samples, n_features)
            The input samples.
            
        Returns:
        --------
        array-like
            The predicted classes.
        """
        if self.tree is None:
            raise Exception("Tree not fitted yet. Call 'fit' first.")
        
        # Convert pandas DataFrame to numpy array if necessary
        if isinstance(X, pd.DataFrame):
            X = X.values
        
        # Convert X to 2D array if it's 1D
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        
        # Predict each sample
        return np.array([self._predict_one(x, self.tree) for x in X])
    
    def print_tree(self, node=None, indent=""):
        """
        Print the tree structure.
        
        Parameters:
        -----------
        node : dict, optional
            The node to start printing from. If None, start from the root.
        indent : str, optional
            The indentation string for the current level.
        """
        if node is None:
            if self.tree is None:
                print("Tree not fitted yet.")
                return
            node = self.tree
        
        if node['type'] == 'leaf':
            class_name = self.class_names[node['class']] if self.class_names is not None else node['class']
            print(f"{indent}Leaf: class={class_name}, samples={node['samples']}")
        else:
            feature_name = self.feature_names[node['feature_idx']] if self.feature_names is not None else f"feature_{node['feature_idx']}"
            print(f"{indent}Node: {feature_name} <= {node['threshold']:.2f}, gain={node['gain']:.4f}, samples={node['samples']}")
            print(f"{indent}  Left branch:")
            self.print_tree(node['left'], indent + "    ")
            print(f"{indent}  Right branch:")
            self.print_tree(node['right'], indent + "    ")


# Function to load the Iris dataset from the UCI repository
def load_iris_data():
    # URL for the Iris dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    
    # Feature names for the Iris dataset
    feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    
    # Class names for the Iris dataset
    class_names = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    
    try:
        # Download the dataset
        response = urllib.request.urlopen(url)
        data = response.read().decode('utf-8')
        
        # Create a pandas DataFrame
        df = pd.read_csv(io.StringIO(data), header=None, names=feature_names + ['class'])
        
        # Convert class names to numeric targets
        class_to_idx = {name: idx for idx, name in enumerate(class_names)}
        df['target'] = df['class'].map(class_to_idx)
        
        return df, feature_names, class_names
    except Exception as e:
        print(f"Error loading Iris dataset: {e}")
        
        # Fallback: Create a simple dataset with the Iris structure
        print("Using hardcoded Iris dataset instead...")
        
        # Define a small sample of the Iris dataset
        data = [
            # Setosa examples
            [5.1, 3.5, 1.4, 0.2, 'Iris-setosa', 0],
            [4.9, 3.0, 1.4, 0.2, 'Iris-setosa', 0],
            [4.7, 3.2, 1.3, 0.2, 'Iris-setosa', 0],
            [4.6, 3.1, 1.5, 0.2, 'Iris-setosa', 0],
            [5.0, 3.6, 1.4, 0.2, 'Iris-setosa', 0],
            # Versicolor examples
            [7.0, 3.2, 4.7, 1.4, 'Iris-versicolor', 1],
            [6.4, 3.2, 4.5, 1.5, 'Iris-versicolor', 1],
            [6.9, 3.1, 4.9, 1.5, 'Iris-versicolor', 1],
            [5.5, 2.3, 4.0, 1.3, 'Iris-versicolor', 1],
            [6.5, 2.8, 4.6, 1.5, 'Iris-versicolor', 1],
            # Virginica examples
            [6.3, 3.3, 6.0, 2.5, 'Iris-virginica', 2],
            [5.8, 2.7, 5.1, 1.9, 'Iris-virginica', 2],
            [7.1, 3.0, 5.9, 2.1, 'Iris-virginica', 2],
            [6.3, 2.9, 5.6, 1.8, 'Iris-virginica', 2],
            [6.5, 3.0, 5.8, 2.2, 'Iris-virginica', 2]
        ]
        
        # Create DataFrame
        df = pd.DataFrame(data, columns=feature_names + ['class', 'target'])
        
        return df, feature_names, class_names


# Function to split the data into training and testing sets
def train_test_split(X, y, test_size=0.3, random_state=None):
    """
    Split the data into training and testing sets.
    
    Parameters:
    -----------
    X : array-like or pandas DataFrame
        Features.
    y : array-like or pandas Series
        Target.
    test_size : float, default=0.3
        Proportion of the dataset to include in the test split.
    random_state : int, default=None
        Controls the shuffling applied to the data.
        
    Returns:
    --------
    X_train, X_test, y_train, y_test
        Splits of X and y.
    """
    # Set random seed if specified
    if random_state is not None:
        np.random.seed(random_state)
    
    # Get the number of samples
    n_samples = len(X)
    
    # Get the number of test samples
    n_test = int(n_samples * test_size)
    
    # Generate random indices for the test set
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    # Split X
    if isinstance(X, pd.DataFrame):
        X_train = X.iloc[train_indices]
        X_test = X.iloc[test_indices]
    else:
        X_train = X[train_indices]
        X_test = X[test_indices]
    
    # Split y
    if isinstance(y, pd.Series):
        y_train = y.iloc[train_indices]
        y_test = y.iloc[test_indices]
    else:
        y_train = y[train_indices]
        y_test = y[test_indices]
    
    return X_train, X_test, y_train, y_test


# Main function to demonstrate the decision tree on the Iris dataset
def main():
    # Load the Iris dataset
    df, feature_names, class_names = load_iris_data()
    
    # Print dataset info
    print("Iris Dataset Info:")
    print(f"Total samples: {len(df)}")
    print(f"Features: {', '.join(feature_names)}")
    print(f"Classes: {', '.join(class_names)}")
    print("\nFirst 5 rows:")
    print(df.head())
    
    # Split the data into features and target
    X = df.drop(['class', 'target'], axis=1)
    y = df['target']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    print(f"\nTraining samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    
    # Create and train the decision tree
    tree = DecisionTreeClassifier(max_depth=3)
    tree.fit(X_train, y_train, feature_names=feature_names, class_names=class_names)
    
    # Print the tree structure
    print("\nDecision Tree Structure:")
    tree.print_tree()
    
    # Make predictions
    y_pred_train = tree.predict(X_train)
    y_pred_test = tree.predict(X_test)
    
    # Calculate accuracy
    train_accuracy = np.mean(y_pred_train == y_train.values)
    test_accuracy = np.mean(y_pred_test == y_test.values)
    
    print(f"\nTraining Accuracy: {train_accuracy:.4f}")
    print(f"Testing Accuracy: {test_accuracy:.4f}")
    
    # Analyze misclassifications
    if isinstance(y_test, pd.Series):
        misclassified_indices = np.where(y_pred_test != y_test.values)[0]
        misclassified = X_test.iloc[misclassified_indices]
        misclassified_true = y_test.iloc[misclassified_indices]
        misclassified_pred = y_pred_test[misclassified_indices]
    else:
        misclassified_indices = np.where(y_pred_test != y_test)[0]
        misclassified = X_test[misclassified_indices]
        misclassified_true = y_test[misclassified_indices]
        misclassified_pred = y_pred_test[misclassified_indices]
    
    if len(misclassified) > 0:
        print(f"\nNumber of misclassified samples: {len(misclassified)}")
        print("\nMisclassified samples:")
        for i in range(len(misclassified)):
            true_class_idx = misclassified_true.iloc[i] if isinstance(misclassified_true, pd.Series) else misclassified_true[i]
            pred_class_idx = misclassified_pred[i]
            true_class = class_names[true_class_idx]
            pred_class = class_names[pred_class_idx]
            print(f"Sample {i+1}: True class = {true_class}, Predicted class = {pred_class}")
            
            if isinstance(misclassified, pd.DataFrame):
                for feature in feature_names:
                    print(f"  {feature}: {misclassified[feature].iloc[i]:.2f}")
            else:
                for j, feature in enumerate(feature_names):
                    print(f"  {feature}: {misclassified[i, j]:.2f}")
    else:
        print("\nNo misclassified samples in the test set!")
    
    # Feature importance analysis
    def get_feature_importance(node, importance_scores):
        """
        Extract feature importance from the tree structure.
        """
        if node['type'] == 'node':
            # Add this node's information gain to the feature's importance
            importance_scores[node['feature_idx']] += node['gain'] * node['samples']
            
            # Recursively process child nodes
            get_feature_importance(node['left'], importance_scores)
            get_feature_importance(node['right'], importance_scores)
    
    # Initialize feature importance scores
    importance_scores = np.zeros(len(feature_names))
    
    # Calculate feature importance
    get_feature_importance(tree.tree, importance_scores)
    
    # Normalize importance scores
    if np.sum(importance_scores) > 0:
        importance_scores = importance_scores / np.sum(importance_scores)
    
    # Print feature importance
    print("\nFeature Importance:")
    for i, (feature, importance) in enumerate(zip(feature_names, importance_scores)):
        print(f"{feature}: {importance:.4f}")
    
    # Return the trained model for further analysis if needed
    return tree, X_test, y_test

if __name__ == "__main__":
    main()

Iris Dataset Info:
Total samples: 150
Features: sepal_length, sepal_width, petal_length, petal_width
Classes: Iris-setosa, Iris-versicolor, Iris-virginica

First 5 rows:
   sepal_length  sepal_width  petal_length  petal_width        class  target
0           5.1          3.5           1.4          0.2  Iris-setosa       0
1           4.9          3.0           1.4          0.2  Iris-setosa       0
2           4.7          3.2           1.3          0.2  Iris-setosa       0
3           4.6          3.1           1.5          0.2  Iris-setosa       0
4           5.0          3.6           1.4          0.2  Iris-setosa       0

Training samples: 105
Testing samples: 45

Decision Tree Structure:
Node: petal_length <= 1.90, gain=0.3121, samples=105
  Left branch:
    Leaf: class=Iris-setosa, samples=31
  Right branch:
    Node: petal_length <= 4.70, gain=0.3551, samples=74
      Left branch:
        Node: petal_width <= 1.50, gain=0.0588, samples=33
          Left branch:
            Leaf: 